## Data Cleaning and Processing

In [47]:
import numpy as np
import pandas as pd

import ast

In [ ]:
# # colsolidate into one dataframe
# df = four_star.append(five_star_clothing, ignore_index=True)
# df = df.append(five_star_not_clothing, ignore_index=True)
# df.shape

# # remove any rows with headers
# df = df.drop(df[df['pattern_id']=='pattern_id'].index)
# # drop douplicates
# df = df.drop_duplicates()

In [84]:
df = pd.read_csv('data/consolidated_patterns.csv', low_memory=False)



# Need to save and reload clean data!! 
df = clean_out_bad_data(df)
df.reset_index(inplace=True, drop=True)

df.to_csv('data/patterns_cleaned.csv', index=False)

# # INCLUDE WEIGHTED AVERAGE SCORE RATHER THAN RATING?